# ESM Experienced Data Scientist Application - Technical Round
- Applicant: **Josha van Spronsen**
- Interviewer(s): **Thiago Fauvrelle**
- Date: **07-03-2024, 11:00-11:40**

### Notebook purpose and structure
This notebook answers the different questions asked during the technical interview. It is structered as follows:
1. Loading necessary packages
1. Importing data
1. (EDA skipped as questions are part of EDA)
1. Questions:
    1. *What are the dimensions of the data set?*
    1. *What is the sample range?*
    1. *Please clean up the dataframe in terminology.*
    1. *Is there missing data?*
    1. *Which country has the most loans to corporations of over EUR 1M (new business)?*
    1. *Please delete countries for which there is more than 50% missing data.*
    1. *Please impute missing values with an approriate method.*

## 1. Load necessary packages

In [1]:
import pandas as pd
import numpy as np

## 2. Importing data

In [2]:
# Read the csv file.
df = pd.read_csv("test.csv")

In [3]:
# Print the first 5 columns.
df.head()

,Unnamed: 0,data.obstime,data.ref_area,data.title,data.obsvalue
0,1,2007-05-01,AT,loans to corporations of up to EUR 1M (new bus...,933.0
1,2,2007-06-01,AT,loans to corporations of up to EUR 1M (new bus...,1099.0
2,3,2007-07-01,AT,loans to corporations of up to EUR 1M (new bus...,1162.0
3,4,2007-08-01,AT,loans to corporations of up to EUR 1M (new bus...,884.0
4,5,2007-09-01,AT,loans to corporations of up to EUR 1M (new bus...,1005.0


## Question 1.A. *What are the dimensions of the data set?*

In [4]:
print(f'There are {df.shape[0]} row observatons and {df.shape[1]} columns.')

There are 11922 row observatons and 5 columns.


## Question 1.B. *What is the sample range?*

In [5]:
print(f'The sample starts at {df["data.obstime"].min()} and runs until {df["data.obstime"].max()}.')

The sample starts at 2005-01-01 and runs until 2024-01-01.


## Question 1.C. *Please clean up the dataframe in terminology.*

In [6]:
# Rename the columns.
df.columns = ['Index','Date','Ref_area','Variable','Obs_value']

# Reset the index and name.
df = df.set_index('Index')

# Show result.
df.head()

,Date,Ref_area,Variable,Obs_value
Index,,,,
1,2007-05-01,AT,loans to corporations of up to EUR 1M (new bus...,933.0
2,2007-06-01,AT,loans to corporations of up to EUR 1M (new bus...,1099.0
3,2007-07-01,AT,loans to corporations of up to EUR 1M (new bus...,1162.0
4,2007-08-01,AT,loans to corporations of up to EUR 1M (new bus...,884.0
5,2007-09-01,AT,loans to corporations of up to EUR 1M (new bus...,1005.0


During the exercise it was suggested to keep the same index and structure as the original data. However, we could also have worked with multiindices (Ref_area,Date) and cast the Variables to different columns. I show both methods for completeness.

In [7]:
# Pivot the DataFrame
df_pivot = df.pivot_table(index=['Ref_area','Date'], columns='Variable', values='Obs_value')
df_pivot.head()

Variable             HICP - Overall index  \
Ref_area Date                               
AT       2007-05-01                   NaN   
         2007-06-01                   1.9   
         2007-07-01                   2.0   
         2007-08-01                   1.7   
         2007-09-01                   2.1   

Variable             loans to corporations of over EUR 1M (new business)  \
Ref_area Date                                                              
AT       2007-05-01                                             5491.0     
         2007-06-01                                             8345.0     
         2007-07-01                                             7370.0     
         2007-08-01                                             6492.0     
         2007-09-01                                             7584.0     

Variable             loans to corporations of up to EUR 1M (new business)  
Ref_area Date                                                              
AT       2007-05-01                                              933.0     
         2007-06-01                                             1099.0     
         2007-07-01                                             1162.0     
         2007-08-01                                              884.0     
         2007-09-01                                             1005.0

## Question 1.D. *Is there missing data?*

In [8]:
# Create series that counts the missing values per column.
srMissing_count = df.isnull().sum(axis=0)

# Name this series 'Missing count'.
srMissing_count.name = 'Missing count'
print(srMissing_count)

Date           0
Ref_area       0
Variable       0
Obs_value    835
Name: Missing count, dtype: int64


In [9]:
# Although redundant for this case, it might be that more columns have missing values:
for col in srMissing_count[srMissing_count>0].index:
    print(f'Column {col} has {srMissing_count[col]} missing values.')

Column Obs_value has 835 missing values.


In [10]:
# In the pivoted dataframe we see more precisely where data is missing.
df_pivot.isnull().sum(axis=0)

Variable
HICP - Overall index                                    104
loans to corporations of over EUR 1M (new business)     636
loans to corporations of up to EUR 1M (new business)    485
dtype: int64

## Question 1.E. *Which country has the most loans to corporations of over EUR 1M (new business)?*

In [11]:
# Extract unique variable names.
sVars = df.Variable.unique()
print(sVars)

['loans to corporations of up to EUR 1M (new business)'
 'loans to corporations of over EUR 1M (new business)'
 'HICP - Overall index']


In [12]:
# Extract rows where we observe 'loans to corporations of over EUR 1M (new business)', 
# then aggregate over reference area's and sort values.
df[df.Variable == sVars[1]].groupby(by=["Ref_area"]).sum().sort_values("Obs_value",ascending=False)

,Obs_value
Ref_area,
U2,35557354.52
DE,11857088.00
IT,4839277.00
ES,4742941.00
BE,3670579.00
FR,3559449.00
NL,1932448.00
AT,1550441.00
LU,552871.00


U2 (euro area changing composition) is of course the largest observation. Second largest value is observed for Germany.

## Question 1.F. *Please delete countries for which there is more than 50% missing data.*

In [13]:
# First let's create a column that indicates (with a 1 instead of True) whether a value is observed.
df['Missing'] = df['Obs_value'].isnull()*1
df.head()

,Date,Ref_area,Variable,Obs_value,Missing
Index,,,,,
1,2007-05-01,AT,loans to corporations of up to EUR 1M (new bus...,933.0,0
2,2007-06-01,AT,loans to corporations of up to EUR 1M (new bus...,1099.0,0
3,2007-07-01,AT,loans to corporations of up to EUR 1M (new bus...,1162.0,0
4,2007-08-01,AT,loans to corporations of up to EUR 1M (new bus...,884.0,0
5,2007-09-01,AT,loans to corporations of up to EUR 1M (new bus...,1005.0,0


In [14]:
# Extract the columns of interest ('Ref_area' and 'Missing'), group by 'Ref_area', 
# and aggregate by taking the mean. Then sort values and multiply by 100 to get percentage values.
df.loc[:,['Ref_area','Missing']].groupby(by=['Ref_area']).mean().sort_values('Missing',ascending=False)*100

,Missing
Ref_area,
LV,40.666667
MT,35.333333
GR,29.333333
LU,20.333333
EE,7.337884
CY,4.104478
SK,2.666667
AT,0.000000
SI,0.000000


There are no countries with more than 50% missing values. We do not have to delete any entries.
Suppose that we would have liked to drop Malta and Latvia (the two highest scoring missing values). The following two (commented out) lines show how that could have been done.

In [15]:
#sCountries_drop = ['MT','LV'];
#df[~df['Ref_area'].isin(sCountries_drop)]

In [16]:
# Create a DataFrame indicating whether each value is None or observed
df_observed = 1*df.pivot_table(index=['Ref_area','Date'], columns='Variable', values='Obs_value', aggfunc=lambda x: x.notnull().any())

In [17]:
# Calculate percentage of missing observations per country and variable.
(1-df_observed.groupby(by=['Ref_area']).mean().sort_values(by=[df_observed.columns.values[1],
                                                            df_observed.columns.values[2]],ascending=True))*100

Variable,HICP - Overall index,loans to corporations of over EUR 1M (new business),loans to corporations of up to EUR 1M (new business)
Ref_area,,,
LV,0.0,68.000000,54.000000
LU,0.0,54.500000,6.500000
MT,0.0,52.000000,54.000000
GR,0.0,39.000000,49.000000
EE,0.0,19.354839,3.500000
CY,0.0,6.707317,6.395349
SK,0.0,4.000000,4.000000
AT,0.0,0.000000,0.000000
BE,0.0,0.000000,0.000000


## Question 1.G. *Please impute missing values with an approriate method.*

In [18]:
# Impute missing value with mean per area and variable.
# Add the value as a different column
df['Obs_value_imputed'] = df.groupby(['Ref_area', 'Variable'])['Obs_value'].transform(lambda x: x.fillna(x.mean()))

In [19]:
# Show that group means are the same.
gdf = df.groupby(by=["Ref_area","Variable"]).mean()
gdf

Obs_value  \
Ref_area Variable                                                            
AT       HICP - Overall index                                     2.699000   
         loans to corporations of over EUR 1M (new busin...    7752.205000   
         loans to corporations of up to EUR 1M (new busi...    1126.615000   
BE       HICP - Overall index                                     2.417000   
         loans to corporations of over EUR 1M (new busin...   18352.895000   
         loans to corporations of up to EUR 1M (new busi...    5889.300000   
CY       HICP - Overall index                                     1.674500   
         loans to corporations of over EUR 1M (new busin...     190.875817   
         loans to corporations of up to EUR 1M (new busi...      73.167702   
DE       HICP - Overall index                                     2.235500   
         loans to corporations of over EUR 1M (new busin...   59285.440000   
         loans to corporations of up to EUR 1M (new busi...   11591.855000   
EE       HICP - Overall index                                     4.466500   
         loans to corporations of over EUR 1M (new busin...     242.150933   
         loans to corporations of up to EUR 1M (new busi...      84.180984   
ES       HICP - Overall index                                     1.972500   
         loans to corporations of over EUR 1M (new busin...   23714.705000   
         loans to corporations of up to EUR 1M (new busi...   16212.870000   
FI       HICP - Overall index                                     2.138000   
         loans to corporations of over EUR 1M (new busin...    2601.050000   
         loans to corporations of up to EUR 1M (new busi...     718.345000   
FR       HICP - Overall index                                     1.899000   
         loans to corporations of over EUR 1M (new busin...   17797.245000   
         loans to corporations of up to EUR 1M (new busi...    8292.850000   
GR       HICP - Overall index                                     1.706000   
         loans to corporations of over EUR 1M (new busin...     964.040984   
         loans to corporations of up to EUR 1M (new busi...     657.186275   
IE       HICP - Overall index                                     1.356000   
         loans to corporations of over EUR 1M (new busin...    2723.740000   
         loans to corporations of up to EUR 1M (new busi...     448.535000   
IT       HICP - Overall index                                     2.084500   
         loans to corporations of over EUR 1M (new busin...   24196.385000   
         loans to corporations of up to EUR 1M (new busi...   14524.100000   
LT       HICP - Overall index                                     4.272000   
         loans to corporations of over EUR 1M (new busin...     269.641300   
         loans to corporations of up to EUR 1M (new busi...      77.761850   
LU       HICP - Overall index                                     2.303000   
         loans to corporations of over EUR 1M (new busin...    6075.505495   
         loans to corporations of up to EUR 1M (new busi...     992.737968   
LV       HICP - Overall index                                     4.204000   
         loans to corporations of over EUR 1M (new busin...     319.194687   
         loans to corporations of up to EUR 1M (new busi...      58.421087   
MT       HICP - Overall index                                     2.208000   
         loans to corporations of over EUR 1M (new busin...      99.370521   
         loans to corporations of up to EUR 1M (new busi...      40.613913   
NL       HICP - Overall index                                     2.350000   
         loans to corporations of over EUR 1M (new busin...    9662.240000   
         loans to corporations of up to EUR 1M (new busi...    1454.795000   
PT       HICP - Overall index                                     1.782000   
         loans to corporations of over EUR 1M (new busin...    1603.459300   
         

In [20]:
# No missing observations in the imputed column anymore. 
df.isnull().sum()

Date                   0
Ref_area               0
Variable               0
Obs_value            835
Missing                0
Obs_value_imputed      0
dtype: int64

We can also interpolate the values.

In [21]:
# Convert the 'Date' column to datetime
df['Date'] = pd.to_datetime(df['Date'])

In [22]:
# Define a function for interpolation within each group
def interpolate_group(group):
    return group.interpolate()

In [23]:
# Sort the DataFrame by 'Ref_area' and 'Variable' for interpolation
df.sort_values(by=['Ref_area', 'Variable', 'Date'], inplace=True)

# Interpolate missing values based on datetime
df['Obs_value_interpolation'] = df.groupby(['Ref_area', 'Variable'])['Obs_value'].apply(interpolate_group)

In [24]:
df

,Date,Ref_area,Variable,Obs_value,Missing,Obs_value_imputed,Obs_value_interpolation
Index,,,,,,,
7923,2007-06-01,AT,HICP - Overall index,1.90,0,1.90,1.90
7924,2007-07-01,AT,HICP - Overall index,2.00,0,2.00,2.00
7925,2007-08-01,AT,HICP - Overall index,1.70,0,1.70,1.70
7926,2007-09-01,AT,HICP - Overall index,2.10,0,2.10,2.10
7927,2007-10-01,AT,HICP - Overall index,2.90,0,2.90,2.90
...,...,...,...,...,...,...,...
7718,2023-08-01,U2,loans to corporations of up to EUR 1M (new bus...,51369.30,0,51369.30,51369.30
7719,2023-09-01,U2,loans to corporations of up to EUR 1M (new bus...,59574.90,0,59574.90,59574.90
7720,2023-10-01,U2,loans to corporations of up to EUR 1M (new bus...,63581.99,0,63581.99,63581.99


In [25]:
# No missing observations in the imputed column anymore. 
df.isnull().sum()

Date                         0
Ref_area                     0
Variable                     0
Obs_value                  835
Missing                      0
Obs_value_imputed            0
Obs_value_interpolation    120
dtype: int64

Interpolation has some missing due to masked observations at the start of the sample. Obsviously we can do better by employing different methods, such as bagging or boosting algorithms, or the MICE algorith by van Buuren.